## Data preparation - directional models (Models B, C, D)

- Tiny Hero dataset source: https://www.kaggle.com/datasets/volodymyrpivoshenko/pixel-characters-dataset
- Data preparation for models B, C, D use data from the 'input_bcd' folder.
- Folders inside 'input_bcd' folder:
    * 0 - Back
    * 1 - Left
    * 2 - Front
    * 3 - Right

- Each pixel art character will have the same file number inside each folder
    * Eg: A character named '12.png' in the '0' folder (Back-facing) will also be named '12.png' in the Left-facing, Front-facing and Right-facing folders

### Directional models

* B - Front(2) to Right(3)
* C - Front(2) to Back(0)
* D - Right(3) to Left(1)

In [1]:
import os
from common_functions import turn_magenta
import common_functions as core

# Variable names for data folders from 'input_bcd' folder
FRONT_FOLDER = core.get_path(core.B1.raw_data_dir, "2")
BACK_FOLDER = core.get_path(core.B1.raw_data_dir, "0")
LEFT_FOLDER = core.get_path(core.B1.raw_data_dir, "1")
RIGHT_FOLDER = core.get_path(core.B1.raw_data_dir, "3")

## Folder structure for model B, C, D data folders

Folder structure for `model_b_data`, `model_c_data` and `model_d_data` folders is the same.

```
model_b_data
    -- input (from FRONT_FOLDER copy files)
        -- train
        -- test
        -- val
    -- output (from RIGHT_FOLDER copy files)
        -- train
        -- test
        -- val
    -- combined (combined input, output images to use for Pix2Pix training)
        -- train
        -- test
        -- val
  ```

### Data content example

Model B converts images from front facing to right facing

- `input` data folder in `model_b_data` contains front facing images
- `output` data folder in `model_b_data` contains right facing images
- `combined` data folder in `model_b_data` contains combined input, output images used by Pix2Pix model

In [2]:
# For loops to create 'input' and 'output' data folders in the
    # 'model_b_data', 'model_c_data', 'model_d_data' directories
for model_directory in [core.B1.prepared_data_dir, core.C1.prepared_data_dir, core.D1.prepared_data_dir]:
    for subdirectory in ['input', 'output']:
        os.makedirs(core.get_path(model_directory, subdirectory), exist_ok=True)

### input, output files for models

- Model B
    - `input` files for model_b_data are in FRONT_FOLDER (folder 2)
    - `output` files for model_b_data are in RIGHT_FOLDER (folder 3)

- Model C
    - `input` files for model_c_data are in FRONT_FOLDER (folder 2)
    - `output` files for model_c_data are in BACK_FOLDER (folder 0)

- Model D
    - `input` files for model_d_data are in RIGHT_FOLDER (folder 3)
    - `output` files for model_d_data are in LEFT_FOLDER (folder 1)

In [3]:
def create_model_data(model_input_data_folder, model_output_data_folder, destination_directory):
    """
    Function to create data for a directional model
    :param model_input_data_folder: example: FRONT_FOLDER
    :param model_output_data_folder: example: RIGHT_FOLDER
    :param destination_directory: example: 'model_b_data'
    """
    files = sorted(os.listdir(model_input_data_folder))
    file_count = len(files)
    # from 0                - test_split_index  -->  train data
    # from test_split_index - val_split_index   -->  test data
    # from val_split_index  - end               -->  validation data
    test_split_index = int(file_count * 0.79)
    val_split_index = int(file_count * 0.9)
    
    # Create subdirectories for train, test, validation
    for subdir in ['train', 'test', 'val']:
        os.makedirs(os.path.join(destination_directory, "input", subdir), exist_ok=True)
        os.makedirs(os.path.join(destination_directory, "output", subdir), exist_ok=True)
        
    # For loop to create data for models
    for idx, file_name in enumerate(files):
        # Select subdirectory for file based on index in for loop
        subdir = "train"
        if test_split_index < idx < val_split_index:
            subdir = "test"
        elif idx >= val_split_index:
            subdir = "val"
        
        # File paths for 'input' folder files for model data folder
        src_file_path_input = os.path.join(model_input_data_folder, file_name)
        dest_file_path_input = os.path.join(destination_directory, 'input', subdir, file_name)
        
        # File paths for 'output' folder files for model data folder
        src_file_path_output = os.path.join(model_output_data_folder, file_name)
        dest_file_path_output = os.path.join(destination_directory, 'output', subdir, file_name)
        
        # Add magenta backgrounds to images before saving them
        turn_magenta(src_file_path_input, dest_file_path_input)
        turn_magenta(src_file_path_output, dest_file_path_output)


In [4]:
# Creating data for model B
create_model_data(FRONT_FOLDER, RIGHT_FOLDER, core.get_path(core.B1.prepared_data_dir))

# Creating data for model C
create_model_data(FRONT_FOLDER, BACK_FOLDER, core.get_path(core.C1.prepared_data_dir))

# Creating data for model D
create_model_data(RIGHT_FOLDER, LEFT_FOLDER, core.get_path(core.D1.prepared_data_dir))

### Combine input, output data folders of a model to one folder so it can be used for Pix2Pix training

In [5]:
MODEL_B1_COMBINED = core.get_path(core.B1.prepared_data_dir, "combined")
MODEL_C1_COMBINED = core.get_path(core.C1.prepared_data_dir, "combined")
MODEL_D1_COMBINED = core.get_path(core.D1.prepared_data_dir, "combined")

In [6]:

# Combined input, output data folders for model B
os.makedirs(MODEL_B1_COMBINED, exist_ok=True)
MODEL_B1_INPUT = core.get_path(core.B1.prepared_data_dir, "input")
MODEL_B1_OUTPUT = core.get_path(core.B1.prepared_data_dir, "output")
core.create_combined_images(f"--fold_A {MODEL_B1_INPUT} --fold_B {MODEL_B1_OUTPUT} --fold_AB {MODEL_B1_COMBINED} --no_multiprocessing")

----------------------------------------------------------
python /Users/tashvit/Documents/GitHub/mmpixagen/thirdparty/pix2pix/datasets/combine_A_and_B.py --fold_A /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_b_data/input/ --fold_B /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_b_data/output/ --fold_AB /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_b_data/combined --no_multiprocessing
[fold_A] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_b_data/input/
[fold_B] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_b_data/output/
[fold_AB] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_b_data/combined
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  True
split = test, use 99/99 images
split = test, number of images = 99
split = train, use 721/721 images
split = train, number of images = 721
split = val, use 92/92 images
split = val, number of images = 92


In [7]:

# Combined input, output data folders for model C
os.makedirs(MODEL_C1_COMBINED, exist_ok=True)
MODEL_C1_INPUT = core.get_path(core.C1.prepared_data_dir, "input")
MODEL_C1_OUTPUT = core.get_path(core.C1.prepared_data_dir, "output")
core.create_combined_images(f"--fold_A {MODEL_C1_INPUT} --fold_B {MODEL_C1_OUTPUT} --fold_AB {MODEL_C1_COMBINED} --no_multiprocessing")

----------------------------------------------------------
python /Users/tashvit/Documents/GitHub/mmpixagen/thirdparty/pix2pix/datasets/combine_A_and_B.py --fold_A /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_c_data/input/ --fold_B /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_c_data/output/ --fold_AB /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_c_data/combined --no_multiprocessing
[fold_A] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_c_data/input/
[fold_B] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_c_data/output/
[fold_AB] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_c_data/combined
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  True
split = test, use 99/99 images
split = test, number of images = 99
split = train, use 721/721 images
split = train, number of images = 721
split = val, use 92/92 images
split = val, number of images = 92


In [8]:

# Combined input, output data folders for model D
os.makedirs(MODEL_D1_COMBINED, exist_ok=True)
MODEL_D1_INPUT = core.get_path(core.D1.prepared_data_dir, "input")
MODEL_D1_OUTPUT = core.get_path(core.D1.prepared_data_dir, "output")
core.create_combined_images(f"--fold_A {MODEL_D1_INPUT} --fold_B {MODEL_D1_OUTPUT} --fold_AB {MODEL_D1_COMBINED} --no_multiprocessing")

----------------------------------------------------------
python /Users/tashvit/Documents/GitHub/mmpixagen/thirdparty/pix2pix/datasets/combine_A_and_B.py --fold_A /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_d_data/input/ --fold_B /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_d_data/output/ --fold_AB /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_d_data/combined --no_multiprocessing
[fold_A] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_d_data/input/
[fold_B] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_d_data/output/
[fold_AB] =  /Users/tashvit/Documents/GitHub/mmpixagen/datasets/model_d_data/combined
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  True
split = test, use 99/99 images
split = test, number of images = 99
split = train, use 721/721 images
split = train, number of images = 721
split = val, use 92/92 images
split = val, number of images = 92
